# User to Query

In [13]:
import pandas as pd


In [14]:
user_df = pd.read_csv('../CampsiteInfo - Sheet1.csv')
user_df.head()


,Date of Query,Name,Rec Area,Weeks in Advance,Number of Nights,Weekends Only,Email
0,11/30/2024,Chad Breece,Yosemite National Park,3,1,0,cmebreece@gmail.com
1,11/30/2024,Chad Breece,Joshua Tree National Park,3,1,0,cmebreece@gmail.com


In [15]:
uniq_users = user_df['Name'].unique()
uniq_users


array(['Chad Breece'], dtype=object)

In [21]:
rec_df = pd.read_csv('rec_area_names.csv')
rec_df.head()

,RecAreaID,RecAreaName
0,4,WestWorld
1,6,Lake Pleasant
2,9,Shadow Mountain Lake
3,10,White Mountains National Recreation Area
4,11,Folsom Lake


In [24]:
uniq_recs = []
for user in uniq_users:
    user_recs = user_df[user_df['Name'] == user]['Rec Area'].unique()
    uniq_recs.append([int(rec_df[rec_df['RecAreaName'] == rec]['RecAreaID'].values[0]) for rec in user_recs])
print(uniq_users, uniq_recs)
print(uniq_users[0], uniq_recs[0])


['Chad Breece'] [[2991, 2782]]
Chad Breece [2991, 2782]
